<a href="https://colab.research.google.com/github/tirthpatel1111/Tops/blob/main/Nueral%20network%20and%20deep%20learning/Assignment/IMDB_RNN_LSTM_GRU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!mkdir -p ~/.kaggle # creating .kaggle folder to store kaggle file
!cp kaggle.json ~/.kaggle/

In [2]:
!kaggle datasets download -d amanbarthwal/imdb-movies-data

Dataset URL: https://www.kaggle.com/datasets/amanbarthwal/imdb-movies-data
License(s): MIT
imdb-movies-data.zip: Skipping, found more recently modified local copy (use --force to force download)


In [3]:
!unzip /content/imdb-movies-data.zip

Archive:  /content/imdb-movies-data.zip
replace imdb-movies-dataset.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: imdb-movies-dataset.csv  


In [4]:
!pip install gensim

In [5]:
import os
import numpy as np
import struct
import pickle
from array import array
%matplotlib inline
import random
import matplotlib.pyplot as plt
import gensim
from gensim.models import Word2Vec
from os.path  import join
import csv
import glob
import imageio
from glob import glob
import shutil
import json
from sklearn.impute import KNNImputer,SimpleImputer
import pandas as pd
import torch
from torch import nn, optim
from torch.autograd import Variable
from torch.utils.data import DataLoader, TensorDataset, Dataset
import torchvision.utils as vutils
from torchvision.io import read_image
from torchvision.transforms.functional import rgb_to_grayscale

import re
import math
import random

from collections import Counter


# from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split


In [6]:
df=pd.read_csv('/content/imdb-movies-dataset.csv')

In [7]:
df.head(5)

,Poster,Title,Year,Certificate,Duration (min),Genre,Rating,Metascore,Director,Cast,Votes,Description,Review Count,Review Title,Review
0,https://m.media-amazon.com/images/M/MV5BYWRkZj...,The Idea of You,2023.0,R,115.0,"Comedy, Drama, Romance",6.4,67.0,Michael Showalter,"Anne Hathaway, Nicholas Galitzine, Ella Rubin,...","28,744","Solène, a 40-year-old single mom, begins an un...",166,Hypocrisy as an idea,"This film, as well as the reaction to it, is a..."
1,https://m.media-amazon.com/images/M/MV5BZGI4NT...,Kingdom of the Planet of the Apes,2023.0,PG-13,145.0,"Action, Adventure, Sci-Fi",7.3,66.0,Wes Ball,"Owen Teague, Freya Allan, Kevin Durand, Peter ...","22,248","Many years after the reign of Caesar, a young ...",183,A phenomenal start to another trilogy!,"I'm a big fan of all the planet of the apes, a..."
2,https://m.media-amazon.com/images/M/MV5BZjIyOT...,Unfrosted,2023.0,PG-13,97.0,"Biography, Comedy, History",5.5,42.0,Jerry Seinfeld,"Isaac Bae, Jerry Seinfeld, Chris Rickett, Rach...","18,401","In 1963 Michigan, business rivals Kellogg's an...",333,not funny,Pretty much the worst criticism you can lay on...
3,https://m.media-amazon.com/images/M/MV5BMjA5Zj...,The Fall Guy,2023.0,PG-13,126.0,"Action, Comedy, Drama",7.3,73.0,David Leitch,"Ryan Gosling, Emily Blunt, Aaron Taylor-Johnso...","38,953",A down-and-out stuntman must find the missing ...,384,Everything you needed and more!,Just got out of the Austin premier at SXSW and...
4,https://m.media-amazon.com/images/M/MV5BNTk1MT...,Challengers,2023.0,R,131.0,"Drama, Romance, Sport",7.7,82.0,Luca Guadagnino,"Zendaya, Mike Faist, Josh O'Connor, Darnell Ap...","32,517","Tashi, a former tennis prodigy turned coach, t...",194,"Watch ""Match Point"" instead",This is a tough one. I liked the concept and t...


In [8]:
col=df.isnull().sum().keys()[df.isnull().sum().values>0]

In [9]:
num_col=[]
for i in col:
  if df[i].dtype == 'float64':
    num_col.append(i)

In [10]:

imputer = KNNImputer(n_neighbors=4)
for i in num_col:
  df[i] = imputer.fit_transform(df[[i]])
# df['Metascore'] = imputer.fit_transform(df[['Metascore']])

In [11]:
df.drop(['Certificate'],axis=1,inplace=True)


In [12]:
df.drop(['Poster'],axis=1,inplace=True)


In [13]:
df.drop(['Cast'],axis=1,inplace=True)
df.drop(['Director'],axis=1,inplace=True)

In [14]:
s=SimpleImputer(strategy='most_frequent')

df['Review Title']=s.fit_transform(df[['Review Title']])[:, 0]


In [15]:
df.shape

(10000, 11)

In [16]:
df.head(5)

,Title,Year,Duration (min),Genre,Rating,Metascore,Votes,Description,Review Count,Review Title,Review
0,The Idea of You,2023.0,115.0,"Comedy, Drama, Romance",6.4,67.0,"28,744","Solène, a 40-year-old single mom, begins an un...",166,Hypocrisy as an idea,"This film, as well as the reaction to it, is a..."
1,Kingdom of the Planet of the Apes,2023.0,145.0,"Action, Adventure, Sci-Fi",7.3,66.0,"22,248","Many years after the reign of Caesar, a young ...",183,A phenomenal start to another trilogy!,"I'm a big fan of all the planet of the apes, a..."
2,Unfrosted,2023.0,97.0,"Biography, Comedy, History",5.5,42.0,"18,401","In 1963 Michigan, business rivals Kellogg's an...",333,not funny,Pretty much the worst criticism you can lay on...
3,The Fall Guy,2023.0,126.0,"Action, Comedy, Drama",7.3,73.0,"38,953",A down-and-out stuntman must find the missing ...,384,Everything you needed and more!,Just got out of the Austin premier at SXSW and...
4,Challengers,2023.0,131.0,"Drama, Romance, Sport",7.7,82.0,"32,517","Tashi, a former tennis prodigy turned coach, t...",194,"Watch ""Match Point"" instead",This is a tough one. I liked the concept and t...


In [17]:
df['Rating']=df['Rating'].round()

In [18]:
def clean_text(t):
  t=t.lower()
  t = re.sub(r"[^a-z0-9'\s]+", " ", t)
  t = re.sub(r"\s+", " ", t).strip()
  return t

In [19]:
from nltk.stem import WordNetLemmatizer
wordNetLemmatizer= WordNetLemmatizer()

In [20]:
cat_col=[]
for i in df.columns:
  if df[i].dtype == 'object':
    cat_col.append(i)

In [21]:
cat_col.remove('Votes')

In [22]:
for i in cat_col:
  df[i]=df[i].astype(str).apply(clean_text)
  # df[i]=df[i].apply(wordNetLemmatizer.lemmatize())

In [23]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [24]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [25]:
from nltk import word_tokenize

In [26]:
for i in cat_col:
  df[i]=df[i].apply(word_tokenize)

In [27]:
for i in cat_col:
  df[i]=df[i].apply(lambda x:[wordNetLemmatizer.lemmatize(word,"v") for word in x])

In [28]:
df["Votes"] = (
    df["Votes"]
    .astype(str)                               # make sure it's string
    .str.replace(",", "", regex=False)         # remove commas
    .replace("nan", None)                      # handle string "nan"
)

df["Votes"] = pd.to_numeric(df["Votes"], errors="coerce").fillna(0).astype(int)




In [29]:
df['combined_text']=df['Title']+df['Genre']+df['Description']+df['Review Title']+df['Review']

In [30]:
df.head()

,Title,Year,Duration (min),Genre,Rating,Metascore,Votes,Description,Review Count,Review Title,Review,combined_text
0,"[the, idea, of, you]",2023.0,115.0,"[comedy, drama, romance]",6.0,67.0,28744,"[sol, ne, a, 40, year, old, single, mom, begin...",[166],"[hypocrisy, as, an, idea]","[this, film, as, well, as, the, reaction, to, ...","[the, idea, of, you, comedy, drama, romance, s..."
1,"[kingdom, of, the, planet, of, the, ape]",2023.0,145.0,"[action, adventure, sci, fi]",7.0,66.0,22248,"[many, years, after, the, reign, of, caesar, a...",[183],"[a, phenomenal, start, to, another, trilogy]","[i, 'm, a, big, fan, of, all, the, planet, of,...","[kingdom, of, the, planet, of, the, ape, actio..."
2,[unfrosted],2023.0,97.0,"[biography, comedy, history]",6.0,42.0,18401,"[in, 1963, michigan, business, rival, kellogg,...",[333],"[not, funny]","[pretty, much, the, worst, criticism, you, can...","[unfrosted, biography, comedy, history, in, 19..."
3,"[the, fall, guy]",2023.0,126.0,"[action, comedy, drama]",7.0,73.0,38953,"[a, down, and, out, stuntman, must, find, the,...",[384],"[everything, you, need, and, more]","[just, get, out, of, the, austin, premier, at,...","[the, fall, guy, action, comedy, drama, a, dow..."
4,[challengers],2023.0,131.0,"[drama, romance, sport]",8.0,82.0,32517,"[tashi, a, former, tennis, prodigy, turn, coac...",[194],"[watch, match, point, instead]","[this, be, a, tough, one, i, like, the, concep...","[challengers, drama, romance, sport, tashi, a,..."


In [31]:
x=df.drop(['Rating'],axis=1)
y=df['Rating']

In [32]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [33]:
df['combined_text'].head()

,combined_text
0,"[the, idea, of, you, comedy, drama, romance, s..."
1,"[kingdom, of, the, planet, of, the, ape, actio..."
2,"[unfrosted, biography, comedy, history, in, 19..."
3,"[the, fall, guy, action, comedy, drama, a, dow..."
4,"[challengers, drama, romance, sport, tashi, a,..."


In [34]:
w2v_params = {
    "vector_size": 100,   # embedding size for Word2Vec
    "window": 5,
    "min_count": 2,
    "workers": 4,
    "epochs": 10,
    "seed": 42
}

In [35]:
w2v=Word2Vec(**w2v_params)

In [36]:
w2v.build_vocab(df['combined_text'])

In [37]:
w2v.train(df['combined_text'], total_examples=w2v.corpus_count, epochs=w2v.epochs)

(23058521, 31557920)

In [38]:
w2v.wv.most_similar('drama')

[('komplex', 0.5768193006515503),
 ('shipboard', 0.5339941382408142),
 ('hhhh', 0.5222442746162415),
 ('wallbanger', 0.5059959292411804),
 ('jagna', 0.5015237331390381),
 ('transatlantic', 0.501417875289917),
 ('backwoodsman', 0.4861021637916565),
 ('gottes', 0.482888787984848),
 ('gamekeeper', 0.4827859401702881),
 ('heinous', 0.4751593768596649)]

In [39]:
def tokenize(s): return s.split()

counter = Counter()
for s in x_train["combined_text"]:
    counter.update(s)

In [40]:
counter

Counter({'the': 137192,
         'magnificent': 129,
         'ambersons': 6,
         'drama': 5009,
         'romance': 1709,
         'spoil': 194,
         'young': 2138,
         'heir': 21,
         'to': 57471,
         'decay': 23,
         'amberson': 3,
         'fortune': 79,
         'come': 3396,
         'between': 1674,
         'his': 13483,
         'widow': 89,
         'mother': 830,
         'and': 72663,
         'man': 2781,
         'she': 5468,
         'have': 22082,
         'always': 1219,
         'love': 4492,
         'well': 4399,
         "'": 4613,
         'slice': 48,
         'look': 3364,
         'at': 9208,
         'downfall': 25,
         'of': 63638,
         'a': 75095,
         'careless': 9,
         'family': 2193,
         'parallel': 67,
         'film': 22739,
         "'s": 27365,
         'treatment': 117,
         'in': 39776,
         '1942': 15,
         'people': 4001,
         'may': 1440,
         'initially': 83,
         'be': 

In [41]:
len(counter)

47218

In [42]:
min_freq = 1
PAD, UNK = "<pad>", "<unk>"
max_vocab = 50000
itos = [PAD, UNK] + [tok for tok, c in counter.most_common() if c >= min_freq][:max_vocab-2]
stoi = {tok:i for i,tok in enumerate(itos)}
VOCAB_SIZE = len(itos)
print("Vocab size:", VOCAB_SIZE)
print("itos",itos)
print("stoi",stoi)

Vocab size: 47220
itos ['<pad>', '<unk>', 'the', 'be', 'a', 'and', 'of', 'to', 'in', 'it', 'i', 'that', 'this', "'s", 'film', 'have', 'as', 'with', 'for', 'movie', 'do', 'but', 'you', 'on', 'his', "n't", 'not', 'he', 'an', 'one', 'by', 'all', 'at', 'make', 'from', 'who', 'they', 'like', 'her', 'see', 'so', 'what', 'just', 'there', 'time', 'or', 'about', 'character', 'out', 'more', 'get', 'if', 'when', 'go', 'their', 'story', 'good', 'some', 'she', 'very', 'up', 'my', 'no', 'drama', 'we', 'will', 'can', 'watch', "'", 'think', 'me', 'love', 'which', 'action', 'would', 'well', 'even', 'than', 'great', 'into', 'really', 'most', 'comedy', 'take', 'people', 'much', 'its', 'know', 'only', 'say', 'life', 'first', 'how', 'also', 'movies', 'find', 'give', 'after', 'because', 'best', 'come', 'other', 'way', 'look', 'them', 'him', 'work', 'two', 'could', 'play', 'end', 'many', 'horror', 'act', 'any', 'man', 'feel', 'too', 'never', 'want', 'where', 'over', 'while', 'then', 'world', 'your', 'here', 

In [43]:
MAX_LEN=700
def encode(toks, max_len=MAX_LEN):
    # toks = tokenize(text)
    idxs = [stoi.get(t, stoi[UNK]) for t in toks][:max_len]
    if len(idxs) < max_len:
        idxs += [stoi[PAD]]*(max_len - len(idxs))
    return np.array(idxs, dtype=np.int64)

In [44]:
len(df['combined_text'].values.max())

894

In [45]:
emb_dim = w2v.vector_size
embedding_matrix = np.random.normal(scale=0.6, size=(VOCAB_SIZE, emb_dim)).astype(np.float32)
for tok, idx in stoi.items():
    if tok in w2v.wv:
        embedding_matrix[idx] = w2v.wv[tok]
embedding_matrix[stoi[PAD]] = np.zeros(emb_dim, dtype=np.float32)
embedding_tensor = torch.tensor(embedding_matrix)

In [46]:
embedding_tensor.shape

torch.Size([47220, 100])

In [47]:
embedding_tensor.shape

torch.Size([47220, 100])

In [48]:
df.shape

(10000, 12)

In [49]:
x_train['combined_text'].shape

(8000,)

In [50]:
class ReviewDataset(Dataset):
    def __init__(self, df_x, df_y):
        # force them into plain Python lists
        self.texts = df_x["combined_text"].tolist()
        self.labels = df_y["Rating"].astype(int).tolist()

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, i):
        return (
            torch.tensor(encode(self.texts[i], MAX_LEN), dtype=torch.long),
            torch.tensor(self.labels[i], dtype=torch.long)
        )


In [51]:
y_train=pd.DataFrame(y_train)
y_test=pd.DataFrame(y_test)

In [52]:
train_loader=DataLoader(ReviewDataset(x_train,y_train),batch_size=64,shuffle=True)
test_loader=DataLoader(ReviewDataset(x_test,y_test),batch_size=64,shuffle=True)

In [53]:
for data,label in train_loader:
  print(data.shape)
  print(label.shape)


torch.Size([64, 700])
torch.Size([64])
torch.Size([64, 700])
torch.Size([64])
torch.Size([64, 700])
torch.Size([64])
torch.Size([64, 700])
torch.Size([64])
torch.Size([64, 700])
torch.Size([64])
torch.Size([64, 700])
torch.Size([64])
torch.Size([64, 700])
torch.Size([64])
torch.Size([64, 700])
torch.Size([64])
torch.Size([64, 700])
torch.Size([64])
torch.Size([64, 700])
torch.Size([64])
torch.Size([64, 700])
torch.Size([64])
torch.Size([64, 700])
torch.Size([64])
torch.Size([64, 700])
torch.Size([64])
torch.Size([64, 700])
torch.Size([64])
torch.Size([64, 700])
torch.Size([64])
torch.Size([64, 700])
torch.Size([64])
torch.Size([64, 700])
torch.Size([64])
torch.Size([64, 700])
torch.Size([64])
torch.Size([64, 700])
torch.Size([64])
torch.Size([64, 700])
torch.Size([64])
torch.Size([64, 700])
torch.Size([64])
torch.Size([64, 700])
torch.Size([64])
torch.Size([64, 700])
torch.Size([64])
torch.Size([64, 700])
torch.Size([64])
torch.Size([64, 700])
torch.Size([64])
torch.Size([64, 700])
tor

In [54]:
y_train['Rating'].value_counts()

,count
Rating,
6.0,3022
7.0,2502
8.0,1212
5.0,861
4.0,269
3.0,73
9.0,43
2.0,15
10.0,2


In [55]:
class RNNModel(nn.Module):
    def __init__(self, vocab_size, emb_dim, dropout=0.2, num_classes=10):
        super().__init__()

        self.emb = nn.Embedding(vocab_size, emb_dim, padding_idx=stoi[PAD])

        self.rnn = nn.ModuleList([
            nn.RNN(emb_dim, 128, batch_first=True),
            nn.RNN(128, 28, batch_first=True),
            nn.RNN(28, 64, batch_first=True)
        ])


        self.sequential = nn.Sequential(
            nn.Dropout(dropout),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):

        x = self.emb(x)


        for rnn_layer in self.rnn:
            x, _ = rnn_layer(x)


        last_hidden = x[:, -1, :]

        out = self.sequential(last_hidden)
        return out

In [56]:
class RNNModel(nn.Module):
    def __init__(self, vocab_size, embed_dim=128, hidden_dim=256,
                 num_layers=2, bidirectional=True, dropout=0.3, pad_idx=0):
        super().__init__()

        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=pad_idx)
        self.embedding.weight.data.copy_(embedding_tensor.to(DEVICE))
        self.rnn = nn.RNN(
            input_size=embed_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            bidirectional=bidirectional,
            dropout=dropout,
            batch_first=True
        )

        direction_factor = 2 if bidirectional else 1
        self.fc = nn.Linear(hidden_dim * direction_factor, 10)  # 10 class output

    def forward(self, x, lengths=None):
        emb = self.embedding(x)
        output, hidden = self.rnn(emb)
        # For bidirectional RNN, concatenate the last layer's forward and backward hidden states
        # hidden shape: (num_layers * num_directions, batch_size, hidden_size)
        # We need the last two for the final layer
        final_hidden = torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1)
        logits = self.fc(final_hidden)
        return logits      # (B, 10)

In [ ]:
class LSTM(nn.Module):
    def __init__(self, vocab_size, embed_dim=128, hidden_dim=256,
                 num_layers=2, bidirectional=True, dropout=0.3, pad_idx=0):
        super().__init__()

        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=pad_idx)
        self.embedding.weight.data.copy_(embedding_tensor.to(DEVICE))
        self.lstm = nn.LSTM(
            input_size=embed_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            bidirectional=bidirectional,
            dropout=dropout,
            batch_first=True
        )

        direction_factor = 2 if bidirectional else 1
        self.fc = nn.Linear(hidden_dim * direction_factor, 10)   # 10 classes

    def forward(self, x, lengths=None):
        emb = self.embedding(x)
        output, (hidden, cell) = self.lstm(emb)
        final_hidden = hidden[-1]
        logits = self.fc(final_hidden)
        return logits     # (B, 10)


In [ ]:
import torch
import torch.nn as nn

class GRU_10Class(nn.Module):
    def __init__(self, vocab_size, embed_dim=128, hidden_dim=256,
                 num_layers=2, bidirectional=True, dropout=0.3, pad_idx=0):
        super().__init__()

        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=pad_idx)
        self.embedding.weight.data.copy_(embedding_tensor.to(DEVICE))
        self.gru = nn.GRU(
            input_size=embed_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            bidirectional=bidirectional,
            dropout=dropout,
            batch_first=True
        )

        direction_factor = 2 if bidirectional else 1
        self.fc = nn.Linear(hidden_dim * direction_factor, 10)  # 10 classes

    def forward(self, x, lengths=None):
        emb = self.embedding(x)
        output, hidden = self.gru(emb)
        final_hidden = hidden[-1]
        logits = self.fc(final_hidden)
        return logits


In [57]:
def train_one_epoch(model, loader, crit, opt, device):
    model.train()
    total_loss = 0.0
    for xb, yb in loader:
        xb, yb = xb.to(device), yb.to(device)
        opt.zero_grad()
        preds = model(xb)

        loss = crit(preds, yb-1)
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        opt.step()
        total_loss += loss.item() * xb.size(0)
    return total_loss / len(loader.dataset)


In [58]:
from sklearn.metrics import accuracy_score, f1_score

@torch.no_grad()
def eval_model(model, loader, device):
    model.eval()
    all_y, all_p = [], []
    for xb, yb in loader:
        xb, yb = xb.to(device), yb.to(device).long()

        # forward pass
        preds = model(xb)                   # shape [batch_size, num_classes]
        pred_labels = preds.argmax(dim=1)+1   # get predicted class (0–9)

        all_y.extend(yb.cpu().numpy().tolist())
        all_p.extend(pred_labels.cpu().numpy().tolist())

    y, p = np.array(all_y), np.array(all_p)
    return {
        "accuracy": accuracy_score(y, p),
        "f1-score": f1_score(y, p, average="weighted"),  # weighted for multi-class
    }


In [59]:
if torch.cuda.is_available():
    DEVICE = torch.device("cuda")
else:
    DEVICE = torch.device("cpu")

In [60]:
def run_experiment(model_class, name, epochs=30, lr=0.01, emb_freeze=False,train_loader=train_loader,val_loader=test_loader):
    print(f"\n=== TRAIN {name} ===")
    model = model_class(VOCAB_SIZE, emb_dim).to(DEVICE)

    # load pretrained embeddings

    if emb_freeze:
        model.embedding.weight.requires_grad = False

    crit = nn.CrossEntropyLoss()
    opt = torch.optim.AdamW(model.parameters(), lr=lr)

    best_rmse = float("inf")
    best_state = None

    for ep in range(1, epochs+1):
        tr_loss = train_one_epoch(model, train_loader, crit, opt, DEVICE)
        val_metrics = eval_model(model, val_loader, DEVICE)
        print(f"Epoch {ep:02d} | loss={tr_loss:.4f} | val_accuracy={val_metrics['accuracy']:.4f} | val_f1={val_metrics['f1-score']:.4f} ")

        if val_metrics["accuracy"] < best_rmse:
            best_rmse = val_metrics["accuracy"]
            best_state = {k:v.cpu().clone() for k,v in model.state_dict().items()}

    # Load best and evaluate on test
    # model.load_state_dict(best_state)
    # model.to(DEVICE)
    # test_metrics = eval_model(model, test_loader, DEVICE)
    # print(f"[{name}] Test -> RMSE={test_metrics['rmse']:.4f}, MAE={test_metrics['mae']:.4f}, R²={test_metrics['r2']:.4f}")

    # ckpt_path = f"{name}_best.pth"
    # torch.save(best_state, ckpt_path)
    # print("Saved", ckpt_path)

    # return name, test_metrics, ckpt_path, model

In [61]:
results = []
for model_class, name in [(RNNModel,"RNN")]:
    results.append(run_experiment(model_class, name))


=== TRAIN RNN ===


KeyboardInterrupt: 